In [1]:
!pip install lightgbm==3.1.1

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import *
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import gc
from tqdm.notebook import tqdm
pd.options.display.max_columns = 100

In [3]:
DATA_DIR = 'data/'

In [4]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')

ID_COL, TARGET_COL = 'ID', 'Top-up Month'
target_mapper = pd.Series({'No Top-up Service': 0,
                           '12-18 Months': 1,
                           '18-24 Months': 2,
                           '24-30 Months': 3,
                           '30-36 Months': 4,
                           '36-48 Months': 5,
                           ' > 48 Months': 6,
 })
target_inv_mapper = pd.Series(index = target_mapper.values, data = target_mapper.index)
train[TARGET_COL] = train[TARGET_COL].map(target_mapper)
ss = test[['ID']]
ss[TARGET_COL] = np.nan

/home/sahil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [5]:
use_cols = ['0', '1', '2', '3', '4', '5', '6']

In [6]:
a = pd.read_csv('oofs_seed_122_nikhil_cv_0.7304.csv')[use_cols]
b = pd.read_csv('oofs_seed_2_nikhil_cv_0.733.csv')[use_cols]
c = pd.read_csv('oofs_seed_777_nikhil_cv_0.7325.csv')[use_cols]
blnd_oofs_nikhil = a * 0.3 + b * 0.35 + c * 0.35
blnd_oofs_nikhil = blnd_oofs_nikhil.values
blnd_oofs_nikhil[blnd_oofs_nikhil[:, 0] < 0.5, 0] = 0
print(f1_score(train[TARGET_COL], blnd_oofs_nikhil.argmax(axis=1), average = 'macro'))


0.7362930258503323


In [7]:
a = pd.read_csv('preds_seed_122_nikhil_cv_0.7304.csv')[use_cols]
b = pd.read_csv('preds_seed_2_nikhil_cv_0.733.csv')[use_cols]
c = pd.read_csv('preds_seed_777_nikhil_cv_0.7325.csv')[use_cols]
blnd_preds_nikhil = a * 0.3 + b * 0.35 + c * 0.35
# blnd_preds_nikhil.round(4).to_csv('3_seeds_blended_preds_cv_0.7355.csv', index=False)

In [8]:
use_cols = [0, 1, 2, 3, 4, 5, 6]

In [9]:
a = pd.read_pickle('test_preds_lgbm_depth_128.pkl')[use_cols]
b = pd.read_pickle('test_preds_lgbm_depth_32.pkl')[use_cols]
c = pd.read_pickle('test_preds_lgbm_depth_192.pkl')[use_cols]
d = pd.read_pickle('xgb_test_preds_682_feats_3_seeds_avg.pkl')[use_cols]
overal_sahil_preds = np.concatenate((a, b, c, d), axis=1)

In [10]:
a = pd.read_pickle('oof_preds_lgbm_depth_128.pkl')[use_cols]
b = pd.read_pickle('oof_preds_lgbm_depth_32.pkl')[use_cols]
c = pd.read_pickle('oof_preds_lgbm_depth_192.pkl')[use_cols]
d = pd.read_pickle('xgb_oof_preds_682_feats_3_seeds_avg.pkl')[use_cols]
overal_sahil_oofs = np.concatenate((a, b, c, d), axis=1)

In [11]:
train2 = overal_sahil_oofs.copy()
test2 = overal_sahil_preds.copy()

In [12]:
np.random.seed(52)
N_SPLITS = 9
folds = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 777)

In [13]:
def lgb_f1_score(y_true, y_pred):
    y_pred = y_pred.reshape(7, -1).T
    y_pred = y_pred.argmax(axis=1) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_pred, average='macro'), True

In [14]:
oofs = np.zeros((len(train), 7))
preds = np.zeros((len(test), 7))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
    
    X_trn, y_trn = train2[trn_idx], train[TARGET_COL].iloc[trn_idx]
    X_val, y_val = train2[val_idx], train[TARGET_COL].iloc[val_idx]
    
    params = {
        'learning_rate': 0.02,#np.random.choice([0.05, 0.06, 0.08, 0.04, 0.1])/5,
        'colsample_bytree': 0.8,#np.random.choice([0.25, 0.5, 0.65, 0.8, 0.95, 0.7, 0.68]),
        'reg_alpha': 0.3,#np.random.choice([0.22, 0.85, 0.65, 0.7, 0.1]),
        'reg_lambda': 0.3,#np.random.choice([0.7, 0.1, 0.22, 0.85, 0.65, 0.05]),
        'max_depth': -1,#np.random.choice([4, 6, 8, 12, -1]),
        'n_estimators': 10000000000,
        'metric': 'None',
        'subsample_for_bin': 100000000000,
        'objective':'binary',
        'num_leaves': 31,
        'min_child_samples': 7,
    }

    
    clf = LGBMClassifier(**params)
    clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], verbose = 50, early_stopping_rounds = 200, eval_metric = lgb_f1_score)
    
    vp = clf.predict_proba(X_val)
    val_score = round(f1_score(y_val, vp.argmax(axis = 1), average = 'macro'), 5)
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict_proba(test2)
    
    oofs[val_idx] = vp
    preds += tp/N_SPLITS

    
oof_score = round(f1_score(train[TARGET_COL], oofs.argmax(axis=1), average = 'macro'), 6)
print(f'\nOOF Auc is : {oof_score}')

Training until validation scores don't improve for 200 rounds
[50]	valid_0's f1: 0.744113
[100]	valid_0's f1: 0.750955
[150]	valid_0's f1: 0.750266
[200]	valid_0's f1: 0.74981
[250]	valid_0's f1: 0.748929
[300]	valid_0's f1: 0.748156
Early stopping, best iteration is:
[100]	valid_0's f1: 0.750955

Fold 1 Val score: 0.75095

Training until validation scores don't improve for 200 rounds
[50]	valid_0's f1: 0.749188
[100]	valid_0's f1: 0.749746
[150]	valid_0's f1: 0.75004
[200]	valid_0's f1: 0.750591
[250]	valid_0's f1: 0.750448
[300]	valid_0's f1: 0.751029
[350]	valid_0's f1: 0.752036
[400]	valid_0's f1: 0.750391
[450]	valid_0's f1: 0.751043
[500]	valid_0's f1: 0.74897
Early stopping, best iteration is:
[346]	valid_0's f1: 0.752605

Fold 2 Val score: 0.7526

Training until validation scores don't improve for 200 rounds
[50]	valid_0's f1: 0.742667
[100]	valid_0's f1: 0.750404
[150]	valid_0's f1: 0.749818
[200]	valid_0's f1: 0.748478
[250]	valid_0's f1: 0.749899
Early stopping, best iterati

In [15]:
blnd_final = oofs * 0.65 + blnd_oofs_nikhil * 0.35
print(f1_score(train[TARGET_COL], blnd_final.argmax(axis=1), average = 'macro'))

0.7549807457366647


In [16]:
blnd_final = preds * 0.65 + blnd_preds_nikhil * 0.35

In [17]:
pred_labels = pd.Series(blnd_final.values.argmax(axis = 1))
preds_actual = pred_labels.map(target_inv_mapper)

In [18]:
ss[TARGET_COL] = preds_actual
ss.to_csv('final_submission.csv', index=False)

/home/sahil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
ss[TARGET_COL].value_counts()

No Top-up Service    13564
 > 48 Months           518
36-48 Months           220
24-30 Months           115
30-36 Months           115
18-24 Months           115
12-18 Months            98
Name: Top-up Month, dtype: int64